<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/agus_bench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar la biblioteca de OpenAI si no está instalada
!pip install openai

import openai
import time
import json
from difflib import SequenceMatcher

# -------------------- Configuración del Modelo y Autenticación -------------------- #

# Define tu clave API de OpenAI aquí. Es recomendable usar variables de entorno para mayor seguridad.
# Puedes subirla a Colab de forma segura o utilizar un gestor de secretos.
openai.api_key = 'TU_CLAVE_API_AQUI'  # Reemplaza con tu clave API

# Define el modelo que deseas evaluar
MODEL_NAME = 'gpt-3.5-turbo'  # Cambia según el modelo que prefieras

# -------------------- Definición de Preguntas y Respuestas Esperadas -------------------- #

# Lista de preguntas para el benchmark
benchmark_questions = [
    {
        "question": "¿Cuál es la capital de Francia?",
        "expected_answer": "París"
    },
    {
        "question": "Resuelve la siguiente operación matemática: 15 * 12.",
        "expected_answer": "180"
    },
    {
        "question": "¿Quién escribió 'Cien Años de Soledad'?",
        "expected_answer": "Gabriel García Márquez"
    }
]

# Número de veces que se preguntará a cada modelo por cada pregunta
NUM_RUNS = 5

# -------------------- Función para Consultar al Modelo -------------------- #

def query_model(question, model=MODEL_NAME, temperature=0):
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "Eres un asistente útil."},
                {"role": "user", "content": question}
            ],
            temperature=temperature,
            max_tokens=150
        )
        answer = response.choices[0].message['content'].strip()
        return answer
    except Exception as e:
        print(f"Error al consultar al modelo: {e}")
        return None

# -------------------- Función para Evaluar la Respuesta -------------------- #

def evaluate_answer(model_answer, expected_answer):
    """
    Evalúa la respuesta del modelo comparándola con la respuesta esperada.
    Utiliza una métrica de similitud básica.
    """
    # Normalizar respuestas
    model_answer_norm = model_answer.lower().strip()
    expected_answer_norm = expected_answer.lower().strip()

    # Comparación directa
    if model_answer_norm == expected_answer_norm:
        return True

    # Comparación basada en similitud
    similarity = SequenceMatcher(None, model_answer_norm, expected_answer_norm).ratio()
    return similarity > 0.8  # Umbral de similitud del 80%

# -------------------- Ejecución del Benchmark -------------------- #

def run_benchmark():
    results = []
    for q in benchmark_questions:
        question = q['question']
        expected = q['expected_answer']
        correct_count = 0
        answers = []

        print(f"\nEvaluando la pregunta: {question}")
        print(f"Respuesta esperada: {expected}")

        for i in range(NUM_RUNS):
            print(f"  Consulta {i+1}/{NUM_RUNS}...")
            answer = query_model(question)
            if answer is None:
                print("    No se pudo obtener una respuesta.")
                continue
            is_correct = evaluate_answer(answer, expected)
            answers.append({
                "run": i+1,
                "answer": answer,
                "is_correct": is_correct
            })
            if is_correct:
                correct_count += 1
            print(f"    Respuesta: {answer} | Correcta: {is_correct}")
            time.sleep(1)  # Espera para evitar exceder los límites de la API

        consistency = correct_count / NUM_RUNS
        results.append({
            "question": question,
            "expected_answer": expected,
            "correct_responses": correct_count,
            "consistency": consistency,
            "answers": answers
        })

    return results

# -------------------- Presentación de Resultados -------------------- #

def display_results(results):
    print("\n\n===== Resultados del Benchmark =====\n")
    for res in results:
        print(f"Pregunta: {res['question']}")
        print(f"Respuesta Esperada: {res['expected_answer']}")
        print(f"Respuestas Correctas: {res['correct_responses']} de {NUM_RUNS}")
        print(f"Consistencia: {res['consistency']*100}%")
        print("Detalles de las respuestas:")
        for ans in res['answers']:
            status = "✅ Correcta" if ans['is_correct'] else "❌ Incorrecta"
            print(f"  - Consulta {ans['run']}: {ans['answer']} | {status}")
        print("\n-----------------------------------\n")

# -------------------- Ejecución Principal -------------------- #

if __name__ == "__main__":
    benchmark_results = run_benchmark()
    display_results(benchmark_results)
